In [34]:
import json
import os
import requests
import time
import urllib
import yaml

### Helper functions

In [35]:
def get_nb_hits(json_response):
    return len(json_response['hits']['hits'])

def get_next_link(json_response):
    return json_response['links'].get('next', None)

def reget(url, params=None, **kwargs):
    """
    Sends a GET request and resends it with increasing delays
    when status code 429 (too many requests) is received.

    :param url: URL for the new :class:`Request` object.
    :param params: (optional) Dictionary, list of tuples or bytes to send
        in the query string for the :class:`Request`.
    :param **kwargs: Optional arguments that ``request`` takes.
    :return: :class:`Response <Response>` object
    :rtype: requests.Response
    """
    if params is None:
        print(f"URL: {url}")
    else:
        print(f"URL: {url}, params:")
        redacted_params = params
        del redacted_params['access_token'] # don't want to leak the token
        print(json.dumps(redacted_params, indent = 4))
    delay = 0.0
    while True:  
        response = requests.get(url, params=params, **kwargs)
        if response.status_code != 429: # not too many requests
            return response
        delay += 2
        print(f"delay: {delay}s to circumvent rate limiting...")
        time.sleep(delay)

### Query with one page per hit

The query string uses [elastic search syntax](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html#query-string-syntax).

In [36]:
query = ""
params = {
    'q': urllib.parse.quote(query),
    #'type': 'publication',
    'type': 'dataset',
    'communities': "coacch-co-designing-the-assessment-of-climate-change-costs-h2020-project",
    'size': 1, # hits per page
    'page': 1,
    'access_token': os.environ['ZENODO_API_TOKEN']
}
response = reget("https://zenodo.org/api/records", params = params)
assert response.status_code == 200 # success
j = response.json()
assert len(j['hits']['hits']) == 1 # yup, one hit on the first page

URL: https://zenodo.org/api/records, params:
{
    "q": "",
    "type": "dataset",
    "communities": "coacch-co-designing-the-assessment-of-climate-change-costs-h2020-project",
    "size": 1,
    "page": 1
}


### Show hit (present on this page, presumably)

Since the page size is 1, we can zero-index the single hit on this page.

In [37]:
print(yaml.dump(j['hits']['hits'][0]))

conceptdoi: 10.5281/zenodo.5541893
conceptrecid: '5541893'
created: '2021-09-30T16:33:25.637302+00:00'
doi: 10.5281/zenodo.5541894
files:
- bucket: 47e4e7fc-084d-4e22-83ef-59337a2f86a9
  checksum: md5:8e2a0bd14e36442fb683b4920a9a8381
  key: COACCH_MetaData_energy_demand.csv
  links:
    self: https://zenodo.org/api/files/47e4e7fc-084d-4e22-83ef-59337a2f86a9/COACCH_MetaData_energy_demand.csv
  size: 1718
  type: csv
- bucket: 47e4e7fc-084d-4e22-83ef-59337a2f86a9
  checksum: md5:16afc8057fc0dd1757dddfa4a072ca37
  key: energy_demand_all_rcp.xlsx
  links:
    self: https://zenodo.org/api/files/47e4e7fc-084d-4e22-83ef-59337a2f86a9/energy_demand_all_rcp.xlsx
  size: 157343
  type: xlsx
id: 5541894
links:
  badge: https://zenodo.org/badge/doi/10.5281/zenodo.5541894.svg
  bucket: https://zenodo.org/api/files/47e4e7fc-084d-4e22-83ef-59337a2f86a9
  conceptbadge: https://zenodo.org/badge/doi/10.5281/zenodo.5541893.svg
  conceptdoi: https://doi.org/10.5281/zenodo.5541893
  doi: https://doi.org/10.

### Define a function to custom-process each hit

In [38]:
def process_hit(hit):
    """
    Process a query hit and generate a ReST page.
    """
    print(f"Title: {hit['metadata']['title']}")

    # Define a templated ReST page for the hit
    # ------ BEGIN TEMPLATE ----------
    page = f"""
.. This file is automaticaly generted. Do not edit.

`{hit['metadata']['title']} <{hit['links']['html']}>`_
{'=' * (len(hit['metadata']['title']) + len(hit['links']['html']) + 6)}

.. image:: {hit['links']['badge']}
   :target: {hit['links']['doi']}

Description:
------------

{hit['metadata']['description']}

Authors:
--------
{'; '.join([creator['name'] for creator in hit['metadata']['creators']])}

.. meta::
   :keywords: {'' if 'keywords' not in hit['metadata'] else ', '.join([keyword for keyword in hit['metadata']['keywords']])}
    """
    # ------ END TEMPLATE ------------

    # Write ReST page, basing the filename on the Zenodo ID
    with open(f"../docs/{params['type']}s/{hit['id']}.rst", "w", encoding = 'utf-8', newline = '\n') as rst:
        rst.write(page)
    return hit['id']

### Process each hit, getting the next page/hit every iteration

In [39]:
# Process initial hit
ids = [process_hit(j['hits']['hits'][0])]
# Process next hits/pages until they exhaust
jn = j
while get_next_link(jn): 
    next_response = reget(get_next_link(jn))
    assert next_response.status_code == 200 # success
    jn = next_response.json()
    ids.append(process_hit(jn['hits']['hits'][0]))
# Check that processed hits equals total hits
assert len(ids) == j['hits']['total']
ids

Title: Climate change impacts on energy demand
URL: https://zenodo.org/api/records/?sort=mostrecent&q=&communities=coacch-co-designing-the-assessment-of-climate-change-costs-h2020-project&type=dataset&page=2&size=1
Title: Supply Chain Shocks due to extreme weather events
URL: https://zenodo.org/api/records/?sort=mostrecent&q=&communities=coacch-co-designing-the-assessment-of-climate-change-costs-h2020-project&type=dataset&page=3&size=1
Title: Riverine Flood Insurance assessment indicators under climate and socio-economic change
URL: https://zenodo.org/api/records/?sort=mostrecent&q=&communities=coacch-co-designing-the-assessment-of-climate-change-costs-h2020-project&type=dataset&page=4&size=1
Title: Time of emergence of climate change impacts
URL: https://zenodo.org/api/records/?sort=mostrecent&q=&communities=coacch-co-designing-the-assessment-of-climate-change-costs-h2020-project&type=dataset&page=5&size=1
Title: Climate induced economic shocks using CLIMRISK
URL: https://zenodo.org/a

[5541894,
 5541327,
 5534190,
 5530237,
 5530146,
 5529888,
 5529732,
 5513871,
 4733499]

### Add datasets to their respective class pages

In [40]:
exec(open('classes.py').read())
written = {}
for c in classes:
    # read the class file
    with open(f"../docs/classes/{c}.rst", "r", encoding = 'utf-8') as class_rst:
        lines = class_rst.readlines()
    # find the toctree
    i = 0
    while lines[i].find(".. toctree::") < 0:
        i += 1
    # find the start of the ToC entries
    while lines[i] != '\n':
        i += 1
    i += 1
    assert i > 2
    start = i
    # remove ToC entries
    while i < len(lines) and lines[i] != '\n':
        lines.pop(i)
    # add new ToC entries
    for id in reversed(ids):
        if id in classes[c] or c =='other' and id not in written:
            written[id] = True
            lines.insert(start, f"   ../datasets/{id}\n")
    # write the index file
    with open(f"../docs/classes/{c}.rst", "w", encoding = 'utf-8', newline = '\n') as class_rst:
        class_rst.writelines(lines)

### List root-level keys and value types of the JSON result dictionary

In [173]:
for key in j:
    print(f"'{key}' {type(j[key])}")

'aggregations' <class 'dict'>
'hits' <class 'dict'>
'links' <class 'dict'>


### Show aggregations (aggregate data over all query hits, presumably)

In [174]:
print(yaml.dump(j['aggregations']))

access_right:
  buckets:
  - doc_count: 25
    key: open
  - doc_count: 1
    key: restricted
  doc_count_error_upper_bound: 0
  sum_other_doc_count: 0
file_type:
  buckets:
  - doc_count: 19
    key: pdf
  - doc_count: 5
    key: csv
  - doc_count: 2
    key: xlsx
  - doc_count: 1
    key: docx
  - doc_count: 1
    key: zip
  doc_count_error_upper_bound: 0
  sum_other_doc_count: 0
keywords:
  buckets:
  - doc_count: 5
    key: COACCH
  - doc_count: 2
    key: agriculture
  - doc_count: 1
    key: CLIMRISK
  - doc_count: 1
    key: CLIMRISK-RIVER
  - doc_count: 1
    key: Climate change mitigation; Electricity; Europe; Risk; Capital costs
  - doc_count: 1
    key: Climate-change impacts, Geology
  - doc_count: 1
    key: Coastal cities
  - doc_count: 1
    key: Damage risk
  - doc_count: 1
    key: ERA-5
  - doc_count: 1
    key: GLOFRIS
  doc_count_error_upper_bound: 0
  sum_other_doc_count: 27
type:
  buckets:
  - doc_count: 19
    key: publication
    subtype:
      buckets:
      -

### Show links (navigation links on this page, presumably)

In [175]:
print(yaml.dump(j['links']))

next: https://zenodo.org/api/records/?sort=mostrecent&q=&communities=coacch-co-designing-the-assessment-of-climate-change-costs-h2020-project&type=dataset&page=2&size=1
self: https://zenodo.org/api/records/?sort=mostrecent&q=&communities=coacch-co-designing-the-assessment-of-climate-change-costs-h2020-project&type=dataset&page=1&size=1



### List root-level keys and value types of the 'hits' sub dictionary

Presumably, this is a list of hits returned on the current page as well as a total number of hits, which could have been classified as an 'aggregation' entry.

In [176]:
for key in j['hits']:
    print(f"'{key}' {type(j['hits'][key])}")

'hits' <class 'list'>
'total' <class 'int'>
